In [1]:
import pandas as pd
import numpy as np

# 读取本地csv文件到dataframe
dfUser = pd.read_csv(r"data/dfUser.csv", sep=',', encoding='utf-8',index_col=0)
dfRel = pd.read_csv(r"data/dfRel.csv", sep=',', encoding='utf-8',index_col=0)
dfTruth=pd.read_csv(r"data/dfTruth.csv",index_col=0)
display(dfUser.head(),dfRel.head(),dfTruth.head())

d:\Anaconda\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,id,targetUser,fName,gender,starUser,mobileUser,homeProvince,homeCity,nFriends,nLike,nPosts,nShares,nAlbums,nShortPosts,nVisits,location,privacy,timeline
0,730637175,0,田,0.0,1.0,0.0,湖北,宜昌市,433.0,NaN,3.0,NaN,2.0,0.0,215.0,黄冈市,1.0,0
1,730665853,0,刘,0.0,1.0,0.0,黑龙江,哈尔滨市,663.0,54.0,0.0,NaN,6.0,170.0,784.0,吉林大学,1.0,0
2,730692985,0,庞,0.0,0.0,0.0,陕西,西安市,59.0,2.0,0.0,NaN,3.0,19.0,298.0,西安市,1.0,0
3,730704085,0,颜,1.0,1.0,0.0,吉林,四平市,349.0,3.0,7.0,NaN,3.0,120.0,406.0,吉林大学,1.0,0
4,730704721,0,乌,0.0,1.0,0.0,陕西,西安市,121.0,3.0,0.0,NaN,0.0,6.0,192.0,陕西师大,1.0,0


,userID,friendID
0,87656499,87656573
1,87656499,87656899
2,87656499,87656949
3,87656499,87657325
4,87656499,87657433


,userID,friendID
0,87702431,87658277
1,87702431,87679913
2,87702431,87697903
3,87702431,87712865
4,87702431,157419785


In [17]:
# loc函数取targetUser==1的用户的id
targetUsers = dfUser.loc[dfUser['targetUser'] == 1, ['id']]

# 关联二度好友
df1 = pd.merge(targetUsers, dfRel, left_on=['id'], right_on=['userID'])
df2 = pd.merge(df1, dfRel, left_on=['friendID'], right_on=['userID'])

# 用group by计算共同好友数，形成分层索引；通过reset_index将series转为dataframe
df3 = df2.groupby(['id','friendID_y'])['friendID_x'].count().reset_index()
# 将结算结果改名为userID, ffID, cn
df3.rename(columns={'id':'userID','friendID_y':'ffID','friendID_x':'cn'}, inplace=True)

# 补充用户信息
df4 = pd.merge(df3, dfUser, left_on='userID', right_on='id')
df5 = pd.merge(df4, dfUser, left_on='ffID', right_on='id')

# 关联好友表，用于标记二度好友是否为好友
df6 = pd.merge(df5, dfRel, how='left', left_on=['userID','ffID'], right_on=['userID','friendID'])
# 将标记改为0,1，并改名为isFriend，数据类型改为int
df6.loc[df6['friendID'].isnull(), ['friendID']]=0
df6.loc[df6['friendID']>0, ['friendID']]=1
df6.rename(columns={'friendID':'isFriend'}, inplace=True)
df6['isFriend']=df6['isFriend'].astype(int)

# 计算指标
df6['s_LHN1']=df6['cn']/(df6['nFriends_x']*df6['nFriends_y'])
print(df6.loc[df6['isFriend']>0].info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46047 entries, 24 to 1205682
Data columns (total 41 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   userID          46047 non-null  int64  
 1   ffID            46047 non-null  int64  
 2   cn              46047 non-null  int64  
 3   id_x            46047 non-null  int64  
 4   targetUser_x    46047 non-null  int64  
 5   fName_x         43772 non-null  object 
 6   gender_x        37782 non-null  float64
 7   starUser_x      43772 non-null  float64
 8   mobileUser_x    43772 non-null  float64
 9   homeProvince_x  29292 non-null  object 
 10  homeCity_x      28998 non-null  object 
 11  nFriends_x      43772 non-null  float64
 12  nLike_x         30101 non-null  float64
 13  nPosts_x        32974 non-null  float64
 14  nShares_x       752 non-null    float64
 15  nAlbums_x       32222 non-null  float64
 16  nShortPosts_x   32222 non-null  float64
 17  nVisits_x       43772 non-nu

In [25]:
pd.unique(df1.loc[:,"userID"]).shape

(242,)

In [18]:
df2

,id,userID_x,friendID_x,userID_y,friendID_y
0,732273541,732273541,536855569,536855569,537319031
1,732273541,732273541,536855569,536855569,596890537
2,732273541,732273541,536855569,536855569,732273541
3,732273541,732273541,537319031,537319031,536855569
4,732273541,732273541,537319031,537319031,538063073
...,...,...,...,...,...
3190656,728915677,728915677,1534863243,1534863243,617680405
3190657,728915677,728915677,1534863243,1534863243,621465217
3190658,728915677,728915677,1534863243,1534863243,623535711
3190659,728915677,728915677,1534863243,1534863243,705836943


In [22]:
pd.unique(df5.loc[:,"userID"])

242

In [5]:
df6

,userID,ffID,cn,id_x,targetUser_x,fName_x,gender_x,starUser_x,mobileUser_x,homeProvince_x,...,nPosts_y,nShares_y,nAlbums_y,nShortPosts_y,nVisits_y,location_y,privacy_y,timeline_y,isFriend,s_LHN1
0,87702431,87656949,1,87702431,1,王,1.0,1.0,0.0,江西,...,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000002
1,87808217,87656949,10,87808217,1,张,1.0,1.0,1.0,陕西,...,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000009
2,139840069,87656949,1,139840069,1,叶,0.0,1.0,0.0,安徽,...,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000002
3,150023465,87656949,1,150023465,1,沈,1.0,1.0,0.0,安徽,...,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000002
4,155712967,87656949,1,155712967,1,吕,1.0,0.0,0.0,河南,...,24.0,NaN,1.0,78.0,6719.0,清华大学,1.0,0,0,0.000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205679,3195846773,3486811249,22,3195846773,1,袁,1.0,1.0,0.0,山西,...,0.0,NaN,0.0,1.0,324.0,吉林大学,1.0,0,1,0.000146
1205680,3195846773,3575901351,17,3195846773,1,袁,1.0,1.0,0.0,山西,...,0.0,NaN,0.0,3.0,277.0,吉林大学,1.0,0,1,0.000143
1205681,3195846773,3818278177,16,3195846773,1,袁,1.0,1.0,0.0,山西,...,3.0,NaN,3.0,7.0,756.0,吉林大学,1.0,0,1,0.000082
1205682,3195846773,4017986037,15,3195846773,1,袁,1.0,1.0,0.0,山西,...,18.0,NaN,0.0,1.0,256.0,吉林大学,1.0,0,1,0.000130


In [6]:
targetUsers

,id
87,732273541
293,736347081
335,737382627
428,739353675
601,742848329
...,...
50958,715867225
51142,721579335
51267,724535251
51441,728351165


In [7]:
# 判断是不是二度好友就是我的所有可能的集合
import numpy as np
for i,ID in enumerate(dfTruth.userID):
    if not np.any(df5.loc[df5.loc[:,"userID"]==ID,"ffID"]==dfTruth.loc[i,"friendID"]):
        print(i,ID)

26 87717679
27 87717679
28 87717933
31 87717933
34 87717933
37 87717933
40 87717933
59 87726597
61 87726597
64 87726597
76 87726597
78 87726597
89 87726597
102 87726597
108 87726597
111 87726597
112 87726597
113 87726597
114 87726597
115 87726597
126 87762563
135 87762563
137 87762563
141 87762563
193 87808217
208 87826451
209 87826451
210 87826451
221 87826451
223 87826451
224 87826451
225 87826451
236 135932485
256 139840069
270 139840069
301 155712967
305 155712967
326 156984681
336 156984681
342 156984681
369 156984681
371 156984681
380 167729505
381 167729505
387 167729505
388 167729505
389 167729505
391 167729505
394 167729505
396 167729505
403 167729505
409 167729505
410 167729505
411 167729505
412 167729505
413 167729505
416 167729505
417 167729505
418 167729505
419 167729505
420 167729505
421 167729505
422 167729505
423 167729505
424 167729505
426 168921263
430 168921263
435 168921263
438 168921263
441 168921263
468 168921263
475 171043555
476 171043555
482 171043555
484 17104

In [8]:
len(dfTruth.userID)

718

In [9]:
dfUser.shape

(51537, 18)

In [10]:
pd.unique(df5.ffID).shape

(51081,)

In [11]:
df3

,userID,ffID,cn
0,87702431,87656949,1
1,87702431,87658277,1
2,87702431,87660311,1
3,87702431,87661375,1
4,87702431,87661377,9
...,...,...,...
1205679,3195846773,4097459911,20
1205680,3195846773,4216998091,1
1205681,3195846773,4281480307,17
1205682,3195846773,4283397531,1


In [12]:
# 关联三度好友
df32 = pd.merge(df3, dfRel, left_on=['ffID'], right_on=['userID'])
df32

,userID_x,ffID,cn,userID_y,friendID
0,87702431,87656949,1,87656949,87656499
1,87702431,87656949,1,87656949,87660311
2,87702431,87656949,1,87656949,87661377
3,87702431,87656949,1,87656949,87671607
4,87702431,87656949,1,87656949,141692389
...,...,...,...,...,...
62586474,3195846773,4097459911,20,4097459911,3486811249
62586475,3195846773,4097459911,20,4097459911,3527762873
62586476,3195846773,4097459911,20,4097459911,3575901351
62586477,3195846773,4097459911,20,4097459911,3633878959


In [14]:
# 判断是不是二度+三度好友就是我的所有可能的集合
import numpy as np
for i,ID in enumerate(dfTruth.userID):
    if not np.any(df5.loc[df5.loc[:,"userID"]==ID,"ffID"]==dfTruth.loc[i,"friendID"]):
        if not np.any(df32.loc[df32.loc[:,"userID_x"]==ID,"friendID"]==dfTruth.loc[i,"friendID"]):
           print(i,ID)

26 87717679
27 87717679
37 87717933
40 87717933
89 87726597
111 87726597
112 87726597
113 87726597
114 87726597
115 87726597
126 87762563
137 87762563
141 87762563
210 87826451
221 87826451
224 87826451
236 135932485
301 155712967
305 155712967
326 156984681
336 156984681
342 156984681
369 156984681
371 156984681
380 167729505
388 167729505
389 167729505
409 167729505
410 167729505
411 167729505
412 167729505
413 167729505
416 167729505
417 167729505
418 167729505
419 167729505
420 167729505
421 167729505
422 167729505
423 167729505
426 168921263
430 168921263
435 168921263
438 168921263
468 168921263
475 171043555
476 171043555
484 171043555
487 174893247
490 174893247
496 174893247
501 174893247
504 174893247
514 174893247
520 174893247
544 176408393
613 182244399
615 182244399
635 182244399
640 182244399
641 182244399
643 182244399
644 182244399
683 185203523


* 二度+三度撑死90%的精度

In [33]:
df32.memory_usage().sum()/(1024**3)

2.797833636403084

## 大思路
### 考虑的模型

### 步骤
1. 看二度好友
   1. 构造人工指标：确定相似度，每个人找到前20个最相似的
   2. 训练模型出来——用干净的数据
   3. 推荐8-9个好友
